# WORK  IN PROGRESS


# Exploratory Data Analysis using Spark and Python  
Now that we have an idea of how to explore some data in Spark, the following content describes how to apply some of those principles to the __Exploratory Data Analysis__ methodology within Data Science. 

__Note:__ The infomration within this document is based on the [Python Tutorials](https://www.codementor.io/python/tutorial) from __Code Mentor__. 


## Getting the Data  
### Getting the Data  
https://data.sfgov.org/Public-Safety/SFPD-Incidents-from-1-January-2003/tmnf-yvry  



In [ ]:
#!wget https://data.sfgov.org/api/views/tmnf-yvry/rows.json?accessType=DOWNLOAD -O ~/downloads/incidents.json
#!wget https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD -O ~/downloads/incidents.csv
!hdfs dfs -put ~/downloads/incidents.json /data/
!hdfs dfs -ls /data/

### Importing the Data in Spark (from CSV)
#### Manual Process  

In [ ]:
!hdfs dfs -put ~/downloads/incidents.csv /data/
!hdfs dfs -ls /data/

In [ ]:
from pyspark.sql.types import *
incidentsFile = sc.textFile("hdfs://master:54310/data/incidents.csv")
incidentsFile.take(5)

Blah blah blah isolate header

In [ ]:
header = incidentsFile.first()
header

In [ ]:
fields = [StructField(field_name, StringType(), True) for field_name in header.split(',')]
fields

#### Using spark-csv  

In [2]:
input_csv = "hdfs://master:54310/data/incidents.csv"
df = sqlContext.read.load(input_csv, format='com.databricks.spark.csv', header='true', infereSchema='true')
#df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("hdfs://master:54310/data/incidents.csv")
#df.printSchema()
df.take(1)

[Row(IncidntNum=u'160190088', Category=u'WARRANTS', Descript=u'ENROUTE TO OUTSIDE JURISDICTION', DayOfWeek=u'Friday', Date=u'03/04/2016', Time=u'23:46', PdDistrict=u'TENDERLOIN', Resolution=u'ARREST, BOOKED', Address=u'GOLDEN GATE AV / HYDE ST', X=u'-122.415508242782', Y=u'37.7816542806076', Location=u'(37.7816542806076, -122.415508242782)', PdId=u'16019008862050')]

#### Using Pandas  

In [1]:
import pandas as pd
pd_csv = pd.read_csv('incidents.csv')
pd_df = sqlContext.createDataFrame(pd_csv)
pd_df.take(1)

[Row(IncidntNum=160190088, Category=u'WARRANTS', Descript=u'ENROUTE TO OUTSIDE JURISDICTION', DayOfWeek=u'Friday', Date=u'03/04/2016', Time=u'23:46', PdDistrict=u'TENDERLOIN', Resolution=u'ARREST, BOOKED', Address=u'GOLDEN GATE AV / HYDE ST', X=-122.415508242782, Y=37.7816542806076, Location=u'(37.7816542806076, -122.415508242782)', PdId=16019008862050)]

### Importing the Data into Spark (from JSON)

## Exploring the Data  

In [ ]:
import urllib
import numpy as np
import pandas as pd
#url = 'https://www.quandl.com/api/v3/datasets/BLSE/CES9000000010.csv'
#file = './data/SFPD_Incidents.csv'
#f = urllib.urlretrieve(url, file)
#df = pd.read_csv(file, index_col = 0, thousands  = ',').T
#df.head(20)



#from pyspark import SparkContext
#from pyspark.sql import SQLContext
#import pandas as pd

#pandas_df = pd.read_csv('hdfs://localhost/data/data.csv')  # assuming the file contains a header
# pandas_df = pd.read_csv('file.csv', names = ['column 1','column 2']) # if no header
#s_df = sql_sc.createDataFrame(pandas_df)

file = sc.textFile('hdfs://master:54310/data/incidents.csv')
file.take(5)
#file.count()

Spark SQL can automatically infer the schema of a JSON dataset and load it as a DataFrame. This conversion can be done using SQLContext.read.json on a JSON file.

Note that the file that is offered as a json file is not a typical JSON file. Each line must contain a separate, self-contained valid JSON object. As a consequence, a regular multi-line JSON file will most often fail.

In [ ]:
df = sqlContext.read.load("hdfs://master:54310/data/incidents.json", format='json')

In [ ]:
df.printSchema()

In [ ]:
input_csv = "hdfs://master:54310/data/incidents.csv"
df = sqlContext.read.load(input_csv, format='com.databricks.spark.csv', header='true', infereSchema='true')
#df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("hdfs://master:54310/data/incidents.csv")
#df.printSchema()
df.take(5)